
# 영화 추천 시스템 



# PROBLEM STATEMENT

- 추천시스템은 영화나 노래등을 추천하는데 사용되며, 주로 관심사나 이용 내역을 기반으로 추천한다.
- 이 노트북에서는, Item-based Collaborative Filtering 으로 추천시스템을 구현한다.

- Dataset MovieLens: https://grouplens.org/datasets/movielens/100k/ 


# STEP #0: LIBRARIES IMPORT


In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
import os

In [44]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/ml_plus/data')

# STEP #1: IMPORT DATASET

### Movie_Id_Titles.csv 파일을 읽으세요.

In [45]:
movie_titles_df = pd.read_csv('Movie_Id_Titles.csv')

In [46]:
movie_titles_df

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


### 'u.data' 파일을 구글드라이브에서 열어보세요.

그러면, 탭으로 구분되어 있고, 맨 위에 컬럼이름이 없습니다.

따라서 컬럼이름을 'user_id', 'item_id', 'rating', 'timestamp' 로 지어주면서 데이터프레임으로 읽어오세요.

In [47]:
movies_rating_df = pd.read_csv('u.data', sep='\t' , names=['user_id', 'item_id', 'rating', 'timestamp' ])

In [48]:
movies_rating_df

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742
...,...,...,...,...
99998,880,476,3,880175444
99999,716,204,5,879795543
100000,276,1090,1,874795795
100001,13,225,2,882399156


### timestamp 컬럼은 필요없으니, movies_rating_df 에서 아예 제거하시오.

In [49]:
movies_rating_df.drop('timestamp' , axis = 1 , inplace=True)

In [50]:
movies_rating_df

,user_id,item_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3
...,...,...,...
99998,880,476,3
99999,716,204,5
100000,276,1090,1
100001,13,225,2


### 두개의 데이터프레임을 합치시오.

### 어떻게 합칠지 모양을 잘 보고 합칩니다.

In [51]:
movie_titles_df.head(2)

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)


In [52]:
movies_rating_df.head(2)

,user_id,item_id,rating
0,0,50,5
1,0,172,5


In [53]:
movies_rating_df = pd.merge(movie_titles_df, movies_rating_df, 
                            on = 'item_id', how='left')

In [54]:
movies_rating_df

,item_id,title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3
...,...,...,...,...
99998,1678,Mat' i syn (1997),863,1
99999,1679,B. Monkey (1998),863,3
100000,1680,Sliding Doors (1998),863,2
100001,1681,You So Crazy (1994),896,3


In [55]:
movie_titles_df.shape

(1682, 2)

In [56]:
movies_rating_df

,item_id,title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3
...,...,...,...,...
99998,1678,Mat' i syn (1997),863,1
99999,1679,B. Monkey (1998),863,3
100000,1680,Sliding Doors (1998),863,2
100001,1681,You So Crazy (1994),896,3


# STEP #2: VISUALIZE DATASET

### 각 영화 제목별로, 별점에 대한 기본통계치(최대,최소,중앙,표준편차,1/4,3/4 값)를 보여주세요.

In [59]:
movies_rating_df.groupby('item_id')['rating'].describe()

,count,mean,std,min,25%,50%,75%,max
item_id,,,,,,,,
1,452.0,3.878319,0.927897,1.0,3.0,4.0,5.0,5.0
2,131.0,3.206107,0.966497,1.0,3.0,3.0,4.0,5.0
3,90.0,3.033333,1.212760,1.0,2.0,3.0,4.0,5.0
4,209.0,3.550239,0.965069,1.0,3.0,4.0,4.0,5.0
5,86.0,3.302326,0.946446,1.0,3.0,3.0,4.0,5.0
...,...,...,...,...,...,...,...,...
1678,1.0,1.000000,NaN,1.0,1.0,1.0,1.0,1.0
1679,1.0,3.000000,NaN,3.0,3.0,3.0,3.0,3.0
1680,1.0,2.000000,NaN,2.0,2.0,2.0,2.0,2.0


In [60]:
# item_id 로 그룹바이하면, 이 값이 인덱스로 표현되므로
# 판다스에서 데이터를 분석할때는, 인덱스가 사람이 해석할수 있는 데이터로
# 나타나도록 해주는게 좋다.

In [62]:
movies_rating_df.groupby('title')['rating'].describe()

,count,mean,std,min,25%,50%,75%,max
title,,,,,,,,
'Til There Was You (1997),9.0,2.333333,1.000000,1.0,2.00,2.0,3.0,4.0
1-900 (1994),5.0,2.600000,1.516575,1.0,1.00,3.0,4.0,4.0
101 Dalmatians (1996),109.0,2.908257,1.076184,1.0,2.00,3.0,4.0,5.0
12 Angry Men (1957),125.0,4.344000,0.719588,2.0,4.00,4.0,5.0,5.0
187 (1997),41.0,3.024390,1.172344,1.0,2.00,3.0,4.0,5.0
...,...,...,...,...,...,...,...,...
Young Guns II (1990),44.0,2.772727,1.008421,1.0,2.00,3.0,3.0,5.0
"Young Poisoner's Handbook, The (1995)",41.0,3.341463,1.237129,1.0,3.00,4.0,4.0,5.0
Zeus and Roxanne (1997),6.0,2.166667,0.983192,1.0,1.25,2.5,3.0,3.0


In [65]:
movies_rating_df.groupby('title')['rating'].describe().sort_values('mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
title,,,,,,,,
They Made Me a Criminal (1939),1.0,5.0,NaN,5.0,5.0,5.0,5.0,5.0
Marlene Dietrich: Shadow and Light (1996),1.0,5.0,NaN,5.0,5.0,5.0,5.0,5.0
"Saint of Fort Washington, The (1993)",2.0,5.0,0.0,5.0,5.0,5.0,5.0,5.0
Someone Else's America (1995),1.0,5.0,NaN,5.0,5.0,5.0,5.0,5.0
Star Kid (1997),3.0,5.0,0.0,5.0,5.0,5.0,5.0,5.0
...,...,...,...,...,...,...,...,...
"Eye of Vichy, The (Oeil de Vichy, L') (1993)",1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0
King of New York (1990),1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0
Touki Bouki (Journey of the Hyena) (1973),1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0


### 각 영화별 별점의 평균을 구하고, 이를 ratings_df_mean 에 저장하시오.

In [68]:
ratings_df_mean = movies_rating_df.groupby('title')['rating'].mean()

In [69]:
ratings_df_mean

title
'Til There Was You (1997)                2.333333
1-900 (1994)                             2.600000
101 Dalmatians (1996)                    2.908257
12 Angry Men (1957)                      4.344000
187 (1997)                               3.024390
                                           ...   
Young Guns II (1990)                     2.772727
Young Poisoner's Handbook, The (1995)    3.341463
Zeus and Roxanne (1997)                  2.166667
unknown                                  3.444444
Á köldum klaka (Cold Fever) (1994)       3.000000
Name: rating, Length: 1664, dtype: float64

### 각 영화별로, 몇개의 데이터가 있는지 구하고, 이를 ratings_df_count 에 저장하시오.

In [71]:
ratings_df_count = movies_rating_df.groupby('title')['rating'].count()

In [72]:
ratings_df_count

title
'Til There Was You (1997)                  9
1-900 (1994)                               5
101 Dalmatians (1996)                    109
12 Angry Men (1957)                      125
187 (1997)                                41
                                        ... 
Young Guns II (1990)                      44
Young Poisoner's Handbook, The (1995)     41
Zeus and Roxanne (1997)                    6
unknown                                    9
Á köldum klaka (Cold Fever) (1994)         1
Name: rating, Length: 1664, dtype: int64

### 두 데이터프레임을 합치세요. 

In [74]:
df1 = ratings_df_mean.to_frame()

In [77]:
df1.columns = [ 'mean']

In [78]:
df1

,mean
title,
'Til There Was You (1997),2.333333
1-900 (1994),2.600000
101 Dalmatians (1996),2.908257
12 Angry Men (1957),4.344000
187 (1997),3.024390
...,...
Young Guns II (1990),2.772727
"Young Poisoner's Handbook, The (1995)",3.341463
Zeus and Roxanne (1997),2.166667


In [81]:
df2 = ratings_df_count.to_frame()

In [83]:
df2.columns = ['count']

In [84]:
ratings_mean_count_df =  df1.join(df2)

In [85]:
ratings_mean_count_df

,mean,count
title,,
'Til There Was You (1997),2.333333,9
1-900 (1994),2.600000,5
101 Dalmatians (1996),2.908257,109
12 Angry Men (1957),4.344000,125
187 (1997),3.024390,41
...,...,...
Young Guns II (1990),2.772727,44
"Young Poisoner's Handbook, The (1995)",3.341463,41
Zeus and Roxanne (1997),2.166667,6


### 컬럼명을 확인하면, 합쳐진 컬럼들이 rating, rating 이라고 되어있습니다. 이를 count, mean 으로 컬럼명을 셋팅하세요.

### mean 으로 히스토그램을 그려보세요.

### count 로 히스토그램을 그려보세요.

### 평균점수가 5점인 영화들은 어떤 영화인지 확인하세요.

### count 가 가장 많은 것부터 정렬하여 100개까지만 보여주세요.

# STEP #3: 영화 하나에 대한, ITEM-BASED COLLABORATIVE FILTERING 수행!

In [ ]:
# movies_rating_df 를 가지고 아래 피봇테이블 합니다.

### 피봇 테이블을 하여, 콜라보레이티브 필터링 포맷으로 변경

### 전체 영화와, 타이타닉 영화의 상관관계 분석을 하면, 타이타닉을 본 사람들에게 상관계수가 높은 영화를 추천하면 된다. corrwith 함수를 이용한다.

### 실습. star wars 를 본 사람들에게 영화를 추천할 것입니다. 5개의 추천 영화 제목을 찾으세요.

힌트 : 먼저 star wars 의 정확한 이름을 검색해서 찾으세요. 그리고 나서 스타워즈를 본 유저의 데이터를 가져와서, 위와 같이 상관관계분석을 합니다.

# STEP#4: 전체 데이터셋에 대한 ITEM-BASED COLLABORATIVE FILTER 를 만들자!

### 나의 별점 정보를 가지고, 영화를 추천해 달라고 할 것이다! 

내 영화 별점 정보는 My_Ratings.csv 파일에 있다. 

In [ ]:
myRatings = 
 

## 이제는 두번째 영화인, 스타워즈도 추천영화 똑같이 가져오자.

## 위의 추천영화 작업을 자동화 하기 위한 파이프라인을 만드시오. 

힌트 : 반복문을 사용하여 비슷한영화에 대한 데이터프레임을 만들고, 이를 아래 빈 데이터프레임에 계속하여 추가하시오. 반복문이 끝나면, 아래 데이터프레임을 wegiht 컬럼으로 정렬하면 됩니다.

In [ ]:
similar_movies_list = pd.DataFrame()